In [1]:
import numpy as np
import math

In [2]:
# applies sigmoid function to the given value
def sigmoid(val):
    return 1/(1+np.exp(-val))

In [3]:
# i_layer: shape - (1,i)
# weights: shape - (i,j)
# return - output: shape - (1,j)
def forward_propagation(i_layer, weights,bias,activation_function):
    output = np.matmul(i_layer,weights)+bias
#     print(output.shape)
    return np.apply_along_axis(activation_function,axis=0,arr=output).reshape(output.shape)

In [4]:
# value: shape - (1,k)
# target: shape - (1,k)
# return: shape - (1,k)
def error_output_layer(value,target):
    return value*(np.ones(value.shape)-value)*(target-value)

In [5]:
# h_value: shape - (1,j)
# weight_h_o: shape - (j,k)
# error_o: shape - (1,k)
# return: shape - (1,j)
def error_hidden_layer(h_value,weight_h_o,error_o):
    return h_value*(np.ones(h_value.shape)-h_value)*np.matmul(error_o,weight_h_o.T)

In [6]:
# l = learning rate
# error: shape - (1,k)
# layer_value: shape - (1,j)
# return: shape - (j,k)
def weight_increment(l,error,layer_value):
    return l*np.matmul(layer_value.T,error)

In [7]:
def neural_net(data,label, hidden_layer,learning_rate,epochs,bias):
    ip_units=data.shape[1]
    output_layer= label.shape[1]
    
    # initialize weights for both layers
    weights_i_h=np.random.rand(ip_units,hidden_layer)
    biases_h=np.zeros((1,hidden_layer),dtype='float64')
#     biases_h=np.random.rand(1,hidden_layer)
    weights_h_o=np.random.rand(hidden_layer,output_layer)
    biases_o=np.zeros((1,output_layer),dtype='float64')
#     biases_o=np.random.rand(1,output_layer)

    for i in range(epochs):
        total_error=0.0
        hidden=[]
        for x,y in zip(data,label):

            # forward propagation
            input_val=x.reshape((1,len(x)))
            target_val=y.reshape((1,len(y)))
            hidden_val=forward_propagation(input_val,weights_i_h,biases_h,sigmoid)
            hidden.append(hidden_val)
            output_val=forward_propagation(hidden_val,weights_h_o,biases_o,sigmoid)

            #calculate total error
            total_error+= np.mean((output_val-target_val)**2)
            
            #backpropagate error
            error_o= error_output_layer(output_val,target_val)
            error_h= error_hidden_layer(hidden_val,weights_h_o,error_o)

            # calculate delta to update weights
            delta_h_o=weight_increment(learning_rate,error_o,hidden_val)
            delta_i_h=weight_increment(learning_rate,error_h,input_val)

            # weights update
            weights_h_o+=delta_h_o
            weights_i_h+=delta_i_h

            #biases_update
            if bias:
                biases_o+=learning_rate*error_o
                biases_h+=learning_rate*error_h
        
        if i%int(epochs/10) is 0:
            print("Epoch-", str(i), ", Error - ", str(total_error))
    print()
    network={}
    network['weight']=[weights_i_h,weights_h_o]
    network['bias']=[biases_h,biases_o]
    return network,hidden

In [8]:
def predict(network,data):
    
    predictions=[]
    
    for x in data:
#         print(x)
        input_val=x.reshape((1,len(x)))
        i=0
        for w,b in zip(network['weight'],network['bias']):
            input_val=forward_propagation(input_val,w,b,sigmoid)
#             print(i,"=",input_val)
            i+=1
        predictions.append(input_val)
    return predictions

In [9]:
def evaluate(model, x, y):
    output = predict(model, x)
    predictions=[j==np.argmax(i)+1 for i,j in zip(output,y)]
    return np.mean(predictions)

### Autoencoder Neural Network

The network is trained to encode and decode the above set of values. The hidden values represent the encoding for the given input.

In [10]:
d=np.zeros((8,8))

In [11]:
for i in range(8):
    d[i][i]=1

In [12]:
# Input for the autoencoder
d

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [13]:
# d: input data
# d: output data
# 3: no. of neurons in the hidden layer
# 5: learning rate
# 5000: epochs
# False: boolean to indicate if we add bias to the network
encoder_net,h_val=neural_net(d,d,3,5,5000,False)

Epoch- 0 , Error -  1.9024195469154666
Epoch- 500 , Error -  0.2239381646062156
Epoch- 1000 , Error -  0.06817561597190125
Epoch- 1500 , Error -  0.061925344707223884
Epoch- 2000 , Error -  0.05896859026884352
Epoch- 2500 , Error -  0.05716115611474606
Epoch- 3000 , Error -  0.05590205829376038
Epoch- 3500 , Error -  0.05495240444669053
Epoch- 4000 , Error -  0.05419693543577444
Epoch- 4500 , Error -  0.053572607546353124



In [14]:
for i in h_val:
    print(i.round(3).astype("str"))

[['0.002' '0.964' '0.998']]
[['0.57' '0.999' '0.002']]
[['0.999' '0.593' '0.001']]
[['0.082' '0.999' '0.373']]
[['0.086' '0.087' '0.077']]
[['0.985' '0.002' '0.998']]
[['0.312' '0.292' '0.999']]
[['0.999' '0.098' '0.366']]


In [15]:
decoded= predict(encoder_net,d)

In [17]:
# The decoded input
for i in decoded:
    print(i.round(1).astype('str'))

[['1.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0']]
[['0.0' '1.0' '0.0' '0.0' '0.1' '0.0' '0.0' '0.0']]
[['0.0' '0.0' '1.0' '0.0' '0.1' '0.0' '0.0' '0.0']]
[['0.0' '0.0' '0.0' '1.0' '0.1' '0.0' '0.0' '0.0']]
[['0.1' '0.1' '0.1' '0.1' '0.4' '0.1' '0.1' '0.1']]
[['0.0' '0.0' '0.0' '0.0' '0.0' '1.0' '0.0' '0.0']]
[['0.0' '0.0' '0.0' '0.0' '0.1' '0.0' '1.0' '0.0']]
[['0.0' '0.0' '0.0' '0.0' '0.1' '0.0' '0.0' '1.0']]


In [18]:
# The network weights that can be used to encode/decode the input
encoder_net

{'weight': [array([[-6.39534545,  3.2904699 ,  6.25659706],
         [ 0.28316891,  6.60604693, -6.27302003],
         [ 6.51223843,  0.37736802, -6.53894118],
         [-2.41885852,  6.5426077 , -0.51814849],
         [-2.36106372, -2.35203975, -2.47723954],
         [ 4.15498852, -6.32787613,  6.19261575],
         [-0.79281709, -0.88754096,  6.58025421],
         [ 6.59764832, -2.21631032, -0.55031704]]),
  array([[-35.8965406 , -10.01651115,   9.5277685 , -29.6061879 ,
           -1.82773186,  -3.23383377, -22.81075552,  12.77597868],
         [ -4.24505482,   9.49087241,  -9.73522425,  12.13113096,
           -1.78835475, -35.93681841, -23.34667679, -29.36059578],
         [  7.9538547 , -35.35004845, -35.61297912, -16.15066554,
           -1.51751209,   7.04879267,  17.62296611, -16.99561661]])],
 'bias': [array([[0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 0.]])]}

## Classifier Square Loss Neural network

A Multi-class classifier neural network that uses square loss and sigmoid activation function.
Wine dataset used for classification

In [19]:
# Reads the dataset from a csv file
def read_csv(filename):
    with open(filename) as file:

        data=[line[:-1].split(',') for line in file.readlines()]
        data=np.array(data).astype('float64')
    return data

In [20]:
# Forms the output dataset to match the network output layer 
# where each output neuron corresponds to a particular class
def make_output_data(data):
    val={ 1:[1.0,0.0,0.0], 2:[0.0,1.0,0.0],3:[0.0,0.0,1.0]}
    label=[val[i] for i in data[:,0].astype('int')]
    return np.array(label)

In [22]:
wine_train=read_csv('train_wine.csv')

In [23]:
wine_test=read_csv('test_wine.csv')

In [24]:
train_output=make_output_data(wine_train)

In [25]:
wine_train=wine_train[:,1:]

In [26]:
# Min-max normalization performed on the training and testing dataset
nmax=np.max(np.vstack((wine_train,wine_test[:,1:])),axis=0)
nmin=np.min(np.vstack((wine_train,wine_test[:,1:])),axis=0)

In [27]:
norm_wine_train=(wine_train - nmin)/(nmax-nmin)

In [28]:
norm_wine_test=(wine_test[:,1:]-nmin)/(nmax-nmin)

In [41]:
# Input Data
# Output data
# No. of neurons in the hidden layer
# Learning rate
# No. of epochs
# Boolean indicating if bias is added to the network
wine_network,wine_hidden=neural_net(norm_wine_train,train_output,4,2,30,True)

Epoch- 0 , Error -  36.277762162275614
Epoch- 3 , Error -  4.482604242341998
Epoch- 6 , Error -  2.2189460138606223
Epoch- 9 , Error -  1.637008941257083
Epoch- 12 , Error -  1.3646482773837654
Epoch- 15 , Error -  1.1461778301482337
Epoch- 18 , Error -  0.9498653592789318
Epoch- 21 , Error -  0.7122773133831778
Epoch- 24 , Error -  0.552270169301544
Epoch- 27 , Error -  0.32804417937479186



In [42]:
# Get predictions on the test data
pred=predict(wine_network,norm_wine_test)

In [43]:
# Evaluate Training and testing accuracy for the network
print("Training Accuracy for wine dataset: ",evaluate(wine_network,norm_wine_train,np.argmax(train_output,axis=1)+1))
print("Testing Accuracy for wine dataset: ",evaluate(wine_network,norm_wine_test,wine_test[:,0]))

Training Accuracy for wine dataset:  0.9470198675496688
Testing Accuracy for wine dataset:  0.9642857142857143
